In [2]:
import os
import json
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
from tqdm import tqdm

### Json에 있는 Segmentation Annotation 추출 및 line coord 교체

In [3]:
json_dir = os.path.join("/opt/ml/data/crosswalk_filtering_closed.json")

with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)
    images = datas["images"]
    annotations = datas["annotations"]

img_df = pd.DataFrame(images)
annos_df = pd.DataFrame(annotations)

In [22]:
img_df.head()

,license,url,file_name,height,width,data_captured,id
0,0,None,MP_SEL_SUR_000033.jpg,1080,1920,None,0
1,0,None,MP_SEL_SUR_000036.jpg,1080,1920,None,1
2,0,None,MP_SEL_SUR_000039.jpg,1080,1920,None,2
3,0,None,MP_SEL_SUR_000066.jpg,1080,1920,None,3
4,0,None,MP_SEL_SUR_000073.jpg,1080,1920,None,4


In [23]:
annos_df.head()

,id,image_id,category_id,bbox,area,segmentation,iscrowd
0,0,0,1,[],0,"[[1579, 16, 1362, 19, 1351, 27, 1302, 39, 1321...",0
1,1,0,1,[],0,"[[751, 64, 434, 286, 0, 655, 4, 798, 354, 799,...",0
2,0,1,1,[],0,"[[339, 0, 316, 3, 2, 171, 0, 1080, 862, 1080, ...",0
3,0,2,1,[],0,"[[0, 0, 0, 632, 347, 661, 1224, 684, 998, 0, 0...",0
4,0,3,1,[],0,"[[170, 0, 1, 80, 0, 746, 424, 756, 1167, 858, ...",0


## line 좌표 뽑기 or polygon area 만들기

In [6]:
def draw_line(seg_data, mask_image):
    coord = []
    for seg_idx in range(0, len(seg_data), 2):
        x, y = seg_data[seg_idx], seg_data[seg_idx+1]
        # mask_image[x, y] = 1
        coord.append([x,y])
    
    line_image = cv2.polylines(mask_image, [np.array(coord)], True, 255, 10)
    #fill_image = cv2.fillPoly(mask_image, [np.array(coord)], 255)
    return line_image, np.argwhere(line_image == 1)

In [ ]:

image_see = []
#anns = sorted(anns, key=lambda idx : len(idx['segmentation'][0]), reverse=False)
new_seg = {}

for idx in tqdm(range(len(set(annos_df["image_id"])))):
    seg_datas = annos_df[annos_df['image_id']==idx]["segmentation"]
    h = img_df[img_df["id"] == idx]["height"].values[0] # image h
    w = img_df[img_df["id"] == idx]["width"].values[0]
    file_name = img_df[img_df['id']==idx]['file_name'].values[0]
    file_name = file_name.split(".")[0]
    
    
    for index, seg_data in seg_datas.items():
        mask_image = np.zeros([h+1, w+1])
        new_image, new_coord= draw_line(seg_data[0], mask_image)
        #plt.imshow(line_image)
        new_seg.update({index: [list(new_coord.flatten().tolist())]})

    #break

In [41]:
print(type(new_coord.flatten().tolist()[0]))

<class 'int'>


In [42]:
s_new_seg = pd.Series(new_seg)

In [43]:
s_new_seg # new series

0       [[11, 1543, 11, 1544, 11, 1545, 11, 1546, 11, ...
1       [[41, 1243, 41, 1244, 41, 1245, 41, 1246, 41, ...
2       [[0, 311, 0, 312, 0, 313, 0, 314, 0, 315, 0, 3...
3       [[0, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 5, 0, 6, 0,...
4       [[0, 157, 0, 158, 0, 159, 0, 160, 0, 161, 0, 1...
                              ...                        
4882    [[0, 533, 0, 534, 0, 535, 0, 536, 0, 537, 0, 5...
4883    [[0, 567, 0, 568, 0, 569, 0, 570, 0, 571, 0, 5...
4884    [[0, 1300, 0, 1301, 0, 1302, 0, 1303, 0, 1304,...
4885    [[23, 1124, 23, 1125, 23, 1126, 23, 1127, 23, ...
4886    [[139, 519, 139, 520, 139, 521, 139, 522, 139,...
Length: 4887, dtype: object

In [44]:
annos_df

,id,image_id,category_id,bbox,area,segmentation,iscrowd
0,0,0,1,[],0,"[[1579, 16, 1362, 19, 1351, 27, 1302, 39, 1321...",0
1,1,0,1,[],0,"[[751, 64, 434, 286, 0, 655, 4, 798, 354, 799,...",0
2,0,1,1,[],0,"[[339, 0, 316, 3, 2, 171, 0, 1080, 862, 1080, ...",0
3,0,2,1,[],0,"[[0, 0, 0, 632, 347, 661, 1224, 684, 998, 0, 0...",0
4,0,3,1,[],0,"[[170, 0, 1, 80, 0, 746, 424, 756, 1167, 858, ...",0
...,...,...,...,...,...,...,...
4882,0,3473,1,[],0,"[[1520, 0, 1782, 544, 1217, 548, 612, 555, 291...",0
4883,0,3474,1,[],0,"[[577, 0, 0, 333, 0, 784, 315, 780, 535, 778, ...",0
4884,0,3475,1,[],0,"[[1920, 0, 1305, 0, 1920, 747, 1920, 0]]",0
4885,0,3476,1,[],0,"[[1920, 38, 1124, 28, 1142, 61, 1169, 98, 1192...",0


In [45]:
annos_new_df = annos_df.drop('segmentation', axis=1)
annos_new_df

,id,image_id,category_id,bbox,area,iscrowd
0,0,0,1,[],0,0
1,1,0,1,[],0,0
2,0,1,1,[],0,0
3,0,2,1,[],0,0
4,0,3,1,[],0,0
...,...,...,...,...,...,...
4882,0,3473,1,[],0,0
4883,0,3474,1,[],0,0
4884,0,3475,1,[],0,0
4885,0,3476,1,[],0,0


In [46]:
annos_new_df.insert(5, "segmentation", s_new_seg) # insert new_series

In [47]:
annos_new_df

,id,image_id,category_id,bbox,area,segmentation,iscrowd
0,0,0,1,[],0,"[[11, 1543, 11, 1544, 11, 1545, 11, 1546, 11, ...",0
1,1,0,1,[],0,"[[41, 1243, 41, 1244, 41, 1245, 41, 1246, 41, ...",0
2,0,1,1,[],0,"[[0, 311, 0, 312, 0, 313, 0, 314, 0, 315, 0, 3...",0
3,0,2,1,[],0,"[[0, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 5, 0, 6, 0,...",0
4,0,3,1,[],0,"[[0, 157, 0, 158, 0, 159, 0, 160, 0, 161, 0, 1...",0
...,...,...,...,...,...,...,...
4882,0,3473,1,[],0,"[[0, 533, 0, 534, 0, 535, 0, 536, 0, 537, 0, 5...",0
4883,0,3474,1,[],0,"[[0, 567, 0, 568, 0, 569, 0, 570, 0, 571, 0, 5...",0
4884,0,3475,1,[],0,"[[0, 1300, 0, 1301, 0, 1302, 0, 1303, 0, 1304,...",0
4885,0,3476,1,[],0,"[[23, 1124, 23, 1125, 23, 1126, 23, 1127, 23, ...",0


In [48]:
test_f = annos_new_df.to_dict('records')

In [49]:
datas["annotations"] = annos_new_df.to_dict('records')

In [50]:
# print(type(datas["annotations"]))
type(datas["annotations"][0]['segmentation'][0][0])

int

In [51]:
with open("crosswalk_filtering_new.json", "w") as json_file:
    json.dump(datas, json_file, indent=4)